In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr

In [2]:
#Load the image
#path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/_original.jpg'
path_to_image = '../../../Data/OxFrei/09_small_palace/_original.jpg'
original = cv2.imread(path_to_image)
sr.show_image(original)

In [4]:
#Convert to grey scale
grayscale = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
sr.show_image(grayscale)

In [13]:
#Binarize according to threshold
threshold = 1
ret, binarized = cv2.threshold(grayscale, thresh=threshold, maxval=255, type=cv2.THRESH_OTSU)
sr.show_image(binarized)

TypeError: Required argument 'maxval' (pos 3) not found

In [6]:
#img_contour, contours, hierarchy 
contour_image = binarized.copy()
img, contours, hierarchy = cv2.findContours(contour_image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contour_image = cv2.drawContours(contour_image, contours, -1, (128, 255, 0), 1)
show_image(contour_image)

In [ ]:
#Find the connected components (similar to bwlabel)
retval, labels = cv2.connectedComponents(binarized[:,:,0]) 
#An ugly transformation to see the CC's in different colors
dim = np.ceil(np.power(retval, 1.0/3))
step = int(np.floor(255/dim))
print dim, step
rvalues = np.arange(int(np.floor(step/2)),256, step)
colormapping = dict([(i,[rvalues[np.mod(i, dim)], 
                         rvalues[np.floor(np.mod(i, dim*dim)/dim)], 
                         rvalues[np.floor(i/dim)]]) for i in xrange(1, retval)])
colormapping[0] = [0,0,0]
colored_labels = np.array([[colormapping[i] for i in x]  for x in labels], dtype='uint8')
show_image(colored_labels)

In [14]:
?cv2.threshold